In [ ]:
import pandas as pd
import json
from collections import defaultdict
import re
import datetime, time

#### Con este bucle obtenemos los datos sobre la inicialización de todos los niveles jugados

In [ ]:
def extraerTiempos(rawData, anomalias):
    erLevel = re.compile(r'\blevel$\b')
    erInitialized = re.compile(r'\binitialized$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erIdLevel = re.compile(r'/')
    
    data = defaultdict(defaultdict)
    anomalias = []

    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        if erLevel.search(obj) : #Si el objeto de la acción es un nivel
            name = evento["actor"]["name"]
            levelCode = erIdLevel.split(evento["object"]["id"])[-1]
            timestamp = evento["timestamp"]
            if erInitialized.search(verb): #Si la acción es inicio o reinicio
                if "result" in evento: #Significa que ha iniciado el nivel desde el menu
                    if levelCode in data[name]:
                        data[name][levelCode].append({"ini" : timestamp, "fin" : ""})
                    else:
                        data[name][levelCode] = [{"ini" : timestamp, "fin" : ""}]
                #else: #-> sería cuando un nivel es reiniciado, por ahora no vamos a tenerlo en cuenta
            elif erCompleted.search(verb):
                if evento["result"]["success"]: #Significa que el nivel se ha completado con éxito
                    if levelCode in data[name]:
                        data[name][levelCode][-1]["fin"] = timestamp
                    else: #Significa que hay un evento de finalizar que no tiene un evento de inicio
                        anomalias.append(evento)
                else:
                    if levelCode in data[name]:
                        data[name][levelCode][-1]["fin"] = None #Porque el jugador sale al menú o cierra el juego
                    else: #Significa que hay un evento de finalizar que no tiene un evento de inicio
                        anomalias.append(evento)
    return data, anomalias              

In [ ]:
#Esta funcion recibe 2 strings de timestamps, los parsea y devuelve la diferencia en horas, minutos, segundos
def parseTimestamps(inicio, final):
    fin = time.mktime(datetime.datetime.strptime(final, "%Y-%m-%dT%H:%M:%S.%fZ").timetuple())
    ini = time.mktime(datetime.datetime.strptime(inicio, "%Y-%m-%dT%H:%M:%S.%fZ").timetuple())

    hours = int((fin - ini)/3600)
    minutes = int((fin - ini)/60)
    seconds = int(fin - ini)
    t = ""
    if hours != 0:
        t += str(hours) + "h:"
        minutes = int((fin - ini)%60)/60
    if minutes != 0:
        t += str(minutes) + "m:"
        seconds = int((fin - ini)%60)
    t += str(seconds) + "s"
    
    return t

In [ ]:
def tiempoPorNiveles(data, anomalias):
    tiemposJugados = defaultdict(defaultdict)
    for player in data:
        for level in data[player]:
            for times in data[player][level]:
                if times["fin"] != None: #Si no se aborto el intento del nivel
                    try:
                        timeDifference = parseTimestamps(times["ini"], times["fin"])
                        if level in tiemposJugados[player]:
                            tiemposJugados[player][level].append(timeDifference)
                        else:
                            tiemposJugados[player][level] = [timeDifference]
                    except ValueError: #Algunos timestamps de finalizacion estan vacios, ¿Tiene que ser asi?
                        anomalias.append({"jugador" : player,
                                  "nivel" : level,
                                  "Timestamp-Inicio" : times["ini"],
                                  "Timestamp-Fin" : times["fin"],
                                  "Descripcion:" : "No se ha podido parsear uno de los 2 timestamps"})
    return tiemposJugados

In [ ]:
JSONFile = open('traces_Articoding_Escolapias.json')
rawData = json.load(JSONFile)
data, anomalias = extraerTiempos(rawData)
JSONFile.close()

pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
#pd.DataFrame(data)

tiempos = tiempoPorNiveles(data, anomalias)
pd.DataFrame(tiempos)